# Retail Sales Analysis

## 1. Project Overview

- **Problem Statement**: Your goal is to analyze sales data to uncover trends, seasonal patterns, and factors influencing sales. This includes understanding how different variables affect sales performance and identifying opportunities for optimization.

- **Scope**: Utilize transactional data (sales records) and inventory data to provide a comprehensive analysis. This may involve examining sales figures, inventory levels, promotions, and other relevant factors.

Data Source: https://www.kaggle.com/datasets/thedevastator/unlock-profits-with-e-commerce-sales-data

## 2. Imports

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

## 3. Data Loading

In [2]:
sales = pd.read_csv('./datasets/Amazon Sale Report.csv')
sales.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_21276\3939810798.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  sales = pd.read_csv('./datasets/Amazon Sale Report.csv')


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [3]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

### Conversion of datatypes

In [4]:
sales['Date'] = pd.to_datetime(sales['Date'])
sales['ship-postal-code'] = sales['ship-postal-code'].astype(object)
sales.drop(columns='Unnamed: 22', inplace=True)
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               128975 non-null  int64         
 1   Order ID            128975 non-null  object        
 2   Date                128975 non-null  datetime64[ns]
 3   Status              128975 non-null  object        
 4   Fulfilment          128975 non-null  object        
 5   Sales Channel       128975 non-null  object        
 6   ship-service-level  128975 non-null  object        
 7   Style               128975 non-null  object        
 8   SKU                 128975 non-null  object        
 9   Category            128975 non-null  object        
 10  Size                128975 non-null  object        
 11  ASIN                128975 non-null  object        
 12  Courier Status      122103 non-null  object        
 13  Qty                 128975 no

C:\Users\USER\AppData\Local\Temp\ipykernel_21276\140846542.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales['Date'] = pd.to_datetime(sales['Date'])


## 4. Data Cleaning

### Zero Quantity

ALERT!!!!

There is an issue observed! There are entries with zero quantities!

In [5]:
sales.groupby('Qty').size()/sales.shape[0]

Qty
0     0.099298
1     0.897693
2     0.002644
3     0.000248
4     0.000070
5     0.000016
8     0.000008
9     0.000008
13    0.000008
15    0.000008
dtype: float64

The **zero quantity** entries comprise about 10% of the current dataset. It could be:
- Canceled Orders
- Free Samples
- Entry Errors
- Returns and Refunds
- Incomplete Transactions


However, considering the goal is to analyse successful sales, we will be dropping the rows

In [6]:
# Drop rows where 'Qty' is zero
sales = sales[sales['Qty'] != 0]

# Verify the changes
print(sales['Qty'].value_counts())

Qty
1     115780
2        341
3         32
4          9
5          2
15         1
9          1
13         1
8          1
Name: count, dtype: int64


In [7]:
sales.isna().sum()

index                     0
Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status            0
Qty                       0
currency                124
Amount                  124
ship-city                28
ship-state               28
ship-postal-code         28
ship-country             28
promotion-ids         36686
B2B                       0
fulfilled-by          83763
dtype: int64

SOrting out the zero quantities also cleared the missing values for the courier services

### Handling missing values

In [8]:
# Creating a series of percentage of missing values per column
missing_values = sales.isna().mean()*100

# View only columns with missing values
missing_values[missing_values > 0]

currency             0.106742
Amount               0.106742
ship-city            0.024103
ship-state           0.024103
ship-postal-code     0.024103
ship-country         0.024103
promotion-ids       31.580125
fulfilled-by        72.105055
dtype: float64

#### Fulfilled-by column

The **fulfilled-by** column has more than 50% of its entries missing. We will inspect the nature of its values first.

In [9]:
# Check the entries in the fulfiled-by column
print('Fulfilled-by column values:', sales['fulfilled-by'].unique())

Fulfilled-by column values: ['Easy Ship' nan]


Since it contains only one value we can let go of the fulfilled-by column with ease

In [10]:
sales.drop(columns = 'fulfilled-by', axis = 1, inplace = True)

missing_values = sales.isna().mean()*100
missing_values[missing_values > 0]

currency             0.106742
Amount               0.106742
ship-city            0.024103
ship-state           0.024103
ship-postal-code     0.024103
ship-country         0.024103
promotion-ids       31.580125
dtype: float64

#### Currency and Amount columns

In [11]:
# Check the entries in the fulfiled-by column
print('Currency column values:', sales['currency'].unique())

Currency column values: ['INR' nan]


It is possible that since the amount of missing values of **Currency** and **Amount** columns are equal they might be null at the same place.

In [12]:
# Checking possibility of similar positions of null values
sales[sales['currency'].isna()][['currency', 'Amount']]

,currency,Amount
937,NaN,NaN
1057,NaN,NaN
3631,NaN,NaN
3632,NaN,NaN
3634,NaN,NaN
...,...,...
118099,NaN,NaN
124616,NaN,NaN
124617,NaN,NaN
124619,NaN,NaN


Since we have confirmed this, we can drop those rows

In [13]:
# Drop all columns with missing values for the currency and the amount
sales.dropna(subset=['currency', 'Amount'], inplace=True)
sales.isna().sum()

index                     0
Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status            0
Qty                       0
currency                  0
Amount                    0
ship-city                28
ship-state               28
ship-postal-code         28
ship-country             28
promotion-ids         36562
B2B                       0
dtype: int64

#### Ship city, state, postal-code, and country

We will employ the same tactic as with currency and amount columns

In [14]:
# Checking possibility of similar positions of null values
sales[sales['ship-city'].isna()][['ship-city', 'ship-state', 'ship-postal-code', 'ship-country']]

,ship-city,ship-state,ship-postal-code,ship-country
1871,NaN,NaN,NaN,NaN
1872,NaN,NaN,NaN,NaN
8752,NaN,NaN,NaN,NaN
11215,NaN,NaN,NaN,NaN
15688,NaN,NaN,NaN,NaN
16787,NaN,NaN,NaN,NaN
18351,NaN,NaN,NaN,NaN
22930,NaN,NaN,NaN,NaN
24986,NaN,NaN,NaN,NaN
30379,NaN,NaN,NaN,NaN


Since we are in a similar situation with these columns, we will also drop the rows affected

In [15]:
# Drop all rows with missing values for the shipping locations
sales.dropna(subset=['ship-city', 'ship-state', 'ship-postal-code', 'ship-country'], inplace=True)
sales.isna().sum()

index                     0
Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status            0
Qty                       0
currency                  0
Amount                    0
ship-city                 0
ship-state                0
ship-postal-code          0
ship-country              0
promotion-ids         36558
B2B                       0
dtype: int64

#### Promotion IDs

In [16]:
# Creating a series of percentage of missing values per column
missing_values = sales.isna().mean()*100

# View only columns with missing values
missing_values[missing_values > 0]

promotion-ids    31.511171
dtype: float64

Since a large portion of this column is missing, we will fill in a placeholder "No Promotion"

In [18]:
sales['promotion-ids'].fillna('No Promotion', inplace=True)

## 5. Exploratory Data Analysis